In [1]:
import tensorflow as tf
import numpy as np
import time
import os
import pickle

In [2]:
import csv
source_data=[]
target_data=[]
with open('protein-secondary-structure/2018-06-06-pdb-intersect-pisces.csv') as file:
    data=csv.reader(file, delimiter=' ')
    next(data, None)
    for row in data:
        source_data.append(row[0].split(',')[2])
        target_data.append(row[0].split(',')[3])
        

In [3]:
'''
def extract_character_vocab(data):
    set_words=set([char for seq in data for char in seq])
    int2vocab={i:word for i, word in enumerate(list(set_words)+["<PAD>"])}
    vocab2int={word:i for i,word in int2vocab.items()}
    return int2vocab, vocab2int
source_int_to_letter, source_letter_to_int = extract_character_vocab(source_data)
#target_int_to_letter, target_letter_to_int = extract_character_vocab(target_data)
'''

'\ndef extract_character_vocab(data):\n    set_words=set([char for seq in data for char in seq])\n    int2vocab={i:word for i, word in enumerate(list(set_words)+["<PAD>"])}\n    vocab2int={word:i for i,word in int2vocab.items()}\n    return int2vocab, vocab2int\nsource_int_to_letter, source_letter_to_int = extract_character_vocab(source_data)\n#target_int_to_letter, target_letter_to_int = extract_character_vocab(target_data)\n'

In [4]:
source_letter_to_int={'I': 0, 'A': 1, 'C': 2, 'V': 3, 'S': 4, 'K': 5, 'T': 6, 'P': 7, 'N': 8, 'L': 9, 'R': 10, 'W': 11, '*': 12, 'F': 13, 'G': 14, 'Q': 15, 'M': 16, 'Y': 17, 'H': 18, 'D': 19, 'E': 20,'<PAD>':21}
source_int_to_letter={0: 'I', 1: 'A', 2: 'C', 3: 'V', 4: 'S', 5: 'K', 6: 'T', 7: 'P', 8: 'N', 9: 'L', 10: 'R', 11: 'W', 12: '*', 13: 'F', 14: 'G', 15: 'Q', 16: 'M', 17: 'Y', 18: 'H', 19: 'D', 20: 'E', 21:'<PAD>'}
target_int_to_letter={0: 'I', 1: 'T', 2: 'E', 3: 'H', 4: 'C', 5: 'B', 6: 'S', 7: 'G', 8:'<PAD>'}
target_letter_to_int={'I': 0, 'T': 1, 'E': 2, 'H': 3, 'C': 4, 'B': 5, 'S': 6, 'G': 7, '<PAD>':8}

In [5]:
# for testing only

import glob
test_sor=[]
test_tar=[]
for file in glob.glob('CB513_RES_nd_its_Secondary_only/CB515/*'):
    with open(file) as f:
        data=f.readlines()
        test_sor.append(data[0])
        test_tar.append(data[1])

        
        
test_sor_=[ val[4:].strip("\n") for val in test_sor ]
test_tar_=[ val[7:].strip("\n") for val in test_tar ]

test_sor_=[''.join(val.split(',')) for val in test_sor_]
test_tar_=[''.join(val.split(',')) for val in test_tar_]

test_seq_id=[[source_letter_to_int.get(letter, source_letter_to_int['*']) for letter in line] for line in test_sor_]
test_tar_id=[[target_letter_to_int.get(letter) for letter in line] for line in test_tar_]


In [64]:
c=0
index=[]
for val in test_tar_id:
    for i in val:
        if i==None:
            index.append(c)
            break
    c+=1

In [65]:
index_=sorted(index, reverse=True)
for idx in index_:
    del test_seq_id[idx]
    del test_tar_id[idx]


In [70]:
test_seq_id=test_seq_id[:-38]
test_tar_id=test_tar_id[:-38]

In [9]:
source_letter_ids_ = [[source_letter_to_int[letter] for letter in line] for line in source_data]
target_letter_ids_ = [[target_letter_to_int[letter] for letter in line] for line in target_data] 

source_letter_ids_=[val for val in source_letter_ids_ if len(val)<300]
target_letter_ids_=[val for val in target_letter_ids_ if len(val)<300]


In [10]:

source_letter_ids_=np.asarray(source_letter_ids_)
#source_letter_ids_=source_letter_ids_/21

target_letter_ids_=np.asarray(target_letter_ids_)

from random import shuffle
idx=[i for i in range(len(source_letter_ids_))]
shuffle(idx)
source_letter_ids_=source_letter_ids_[idx,]
target_letter_ids_=target_letter_ids_[idx,]

source_letter_ids=[]
target_letter_ids=[]
for val in source_letter_ids_:source_letter_ids.append(val)
for val in target_letter_ids_:target_letter_ids.append(val)

In [11]:

source_letter_ids=source_letter_ids[:-12]
target_letter_ids=target_letter_ids[:-12]


In [12]:
epochs=25
lstm_units=32
num_layers=2
batch_size=32
learn_rate=1e-3
#embed_dim=50
display_step=100



In [13]:
tf.reset_default_graph()
train_graph=tf.Graph()

def makeCell(lstm_units):
    cell=tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.LSTMCell(lstm_units, initializer=tf.random_uniform_initializer(-1, 1, seed=2)), output_keep_prob=0.75)
    return cell

with train_graph.as_default():
    with tf.device('/gpu:0'):
        x=tf.placeholder(tf.int32, [None,None])
        y=tf.placeholder(tf.int32, [None,None])
        lr=tf.placeholder(tf.float32)
        y_onehot=tf.one_hot(y, len(target_letter_to_int))
        #x_onehot=tf.one_hot(x, len(source_letter_to_int))
        #rev_x=tf.reverse(x, [-1])
        embed=tf.contrib.layers.embed_sequence(x, len(source_letter_to_int), 50)
        print(embed)

        conv3_1=tf.layers.conv1d(embed,8, 3, padding='same')
        conv3_2=tf.layers.conv1d(conv3_1,16, 3, padding='same')
        conv3_2=tf.nn.dropout(conv3_2, keep_prob=0.65)
        conv3_3=tf.layers.conv1d(conv3_2,32, 3, padding='same')
        conv3=tf.nn.dropout(conv3_3, keep_prob=0.7)
        
        conv5_1=tf.layers.conv1d(embed,8, 5, padding='same')
        conv5_2=tf.layers.conv1d(conv5_1,16, 5, padding='same')
        conv5_2=tf.nn.dropout(conv5_2, keep_prob=0.65)
        conv5_3=tf.layers.conv1d(conv5_2, 32, 5, padding='same')
        conv5_3=tf.nn.dropout(conv5_3, keep_prob=0.7)
        conv5_4=tf.layers.conv1d(embed,8, 2, padding='same')
        conv5_5=tf.layers.conv1d(conv5_4, 16, 2, padding='same')
        conv5_5=tf.nn.dropout(conv5_5, keep_prob=0.6)
        conv5=tf.concat([conv5_3, conv5_5], 2)

        conv7_1=tf.layers.conv1d(embed,8, 7, padding='same')
        conv7_2=tf.layers.conv1d(conv7_1,16, 7, padding='same')
        conv7_2=tf.nn.dropout(conv7_2, keep_prob=0.65)
        conv7_3=tf.layers.conv1d(conv7_2, 32, 7, padding='same')
        conv7_3=tf.nn.dropout(conv7_3, keep_prob=0.7)
        conv7_4=tf.layers.conv1d(embed,8, 3, padding='same')
        conv7_5=tf.layers.conv1d(conv7_4, 16, 3, padding='same')
        conv7_6=tf.layers.conv1d(conv7_5, 32, 3, padding='same')
        conv7_6=tf.nn.dropout(conv7_6, keep_prob=0.65)
        conv7=tf.concat([conv7_3, conv7_6], 2)

        conv9_1=tf.layers.conv1d(embed,8, 9, padding='same')
        conv9_2=tf.layers.conv1d(conv9_1,16, 9, padding='same')
        conv9_2=tf.nn.dropout(conv9_2, keep_prob=0.65)
        conv9_3=tf.layers.conv1d(conv9_2, 32, 9, padding='same')
        conv9_3=tf.nn.dropout(conv9_3, keep_prob=0.7)
        conv9_4=tf.layers.conv1d(embed,8, 3, padding='same')
        conv9_5=tf.layers.conv1d(conv9_4, 16, 3, padding='same')
        conv9_5=tf.nn.dropout(conv9_5, keep_prob=0.65)
        conv9_6=tf.layers.conv1d(conv9_5, 32, 3, padding='same')
        conv9_7=tf.layers.conv1d(conv9_6, 32, 3, padding='same')
        conv9_7=tf.nn.dropout(conv9_7, keep_prob=0.6)
        conv9=tf.concat([conv9_3, conv9_7], 2)

        conv11_1=tf.layers.conv1d(embed,8, 11, padding='same')
        conv11_2=tf.layers.conv1d(conv11_1,16, 11, padding='same')
        conv11_2=tf.nn.dropout(conv11_2, keep_prob=0.65)
        conv11_3=tf.layers.conv1d(conv11_2, 32, 11, padding='same')
        conv11_3=tf.nn.dropout(conv11_3, keep_prob=0.7)
        conv11_4=tf.layers.conv1d(embed,8, 3, padding='same')
        conv11_5=tf.layers.conv1d(conv11_4, 16, 3, padding='same')
        conv11_6=tf.layers.conv1d(conv11_5, 16, 3, padding='same')
        conv11_6=tf.nn.dropout(conv11_6, keep_prob=0.65)
        conv11_7=tf.layers.conv1d(conv11_6, 32, 3, padding='same')
        conv11_8=tf.layers.conv1d(conv11_7, 32, 3, padding='same')
        conv11_8=tf.nn.dropout(conv11_8, keep_prob=0.5)
        conv11=tf.concat([conv11_3, conv11_8], 2)
   
        conv_concat=tf.concat([conv3, conv5, conv7, conv9, conv11], 2)
        print(conv_concat)
        
        fw_cell=tf.contrib.rnn.MultiRNNCell([makeCell(lstm_units) for _ in range(num_layers)])
        bw_cell=tf.contrib.rnn.MultiRNNCell([makeCell(lstm_units) for _ in range(num_layers)])

        fw_initial_state=fw_cell.zero_state(batch_size, tf.float32)
        bw_initial_state=bw_cell.zero_state(batch_size, tf.float32)

  
        out, final_state=tf.nn.bidirectional_dynamic_rnn(fw_cell, bw_cell, conv_concat, initial_state_fw=fw_initial_state, initial_state_bw=bw_initial_state)
        out=tf.concat(out, 2)
        print(out)

    logits=tf.contrib.layers.fully_connected(out,len(target_letter_to_int), activation_fn=tf.nn.softmax)
    with tf.device('/gpu:0'):
        loss=tf.reduce_mean(tf.losses.log_loss(predictions=logits, labels=y_onehot))
        optm=tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)
        
    tf.summary.scalar("loss", loss)
    model_vars=tf.trainable_variables()
  
    saver=tf.train.Saver()

Tensor("EmbedSequence/embedding_lookup:0", shape=(?, ?, 50), dtype=float32, device=/device:GPU:0)
Tensor("concat_4:0", shape=(?, ?, 272), dtype=float32, device=/device:GPU:0)
Tensor("concat_5:0", shape=(32, ?, 64), dtype=float32, device=/device:GPU:0)


In [14]:
def get_summary(vars_):
    tf.contrib.slim.model_analyzer.analyze_vars(vars_, print_info=True)
    
get_summary(model_vars)

---------
Variables: name (type shape) [size]
---------
EmbedSequence/embeddings:0 (float32_ref 22x50) [1100, bytes: 4400]
conv1d/kernel:0 (float32_ref 3x50x8) [1200, bytes: 4800]
conv1d/bias:0 (float32_ref 8) [8, bytes: 32]
conv1d_1/kernel:0 (float32_ref 3x8x16) [384, bytes: 1536]
conv1d_1/bias:0 (float32_ref 16) [16, bytes: 64]
conv1d_2/kernel:0 (float32_ref 3x16x32) [1536, bytes: 6144]
conv1d_2/bias:0 (float32_ref 32) [32, bytes: 128]
conv1d_3/kernel:0 (float32_ref 5x50x8) [2000, bytes: 8000]
conv1d_3/bias:0 (float32_ref 8) [8, bytes: 32]
conv1d_4/kernel:0 (float32_ref 5x8x16) [640, bytes: 2560]
conv1d_4/bias:0 (float32_ref 16) [16, bytes: 64]
conv1d_5/kernel:0 (float32_ref 5x16x32) [2560, bytes: 10240]
conv1d_5/bias:0 (float32_ref 32) [32, bytes: 128]
conv1d_6/kernel:0 (float32_ref 2x50x8) [800, bytes: 3200]
conv1d_6/bias:0 (float32_ref 8) [8, bytes: 32]
conv1d_7/kernel:0 (float32_ref 2x8x16) [256, bytes: 1024]
conv1d_7/bias:0 (float32_ref 16) [16, bytes: 64]
conv1d_8/kernel:0 (flo

In [15]:
def pad_sentence_batch(sentences, pad_int_id):
    max_len=max([len(sentence) for sentence in sentences])
    return [sentence + [pad_int_id]*(max_len-len(sentence)) for sentence in sentences]

def get_batches(sources, targets, source_pad_int, target_pad_int, batch_size):
    
    for i in range(0, len(sources),batch_size):
        
        end=i+batch_size
        source_batch=sources[i:end]
        target_batch=targets[i:end]
        source_batch_pad=np.array(pad_sentence_batch(source_batch, source_pad_int))
        #source_batch_pad=source_batch_pad/21
        #source_batch_pad=[2.*(a - np.min(a))/np.ptp(a)-1 for a in source_batch_pad]
        #source_batch_pad=[arr[:, np.newaxis] for arr in source_batch_pad]
        target_batch_pad=np.array(pad_sentence_batch(target_batch, target_pad_int))

        yield source_batch_pad, target_batch_pad

In [16]:
valid_source, valid_target=next(get_batches(source_letter_ids, target_letter_ids, source_letter_to_int['<PAD>'], target_letter_to_int['<PAD>'],batch_size))

In [17]:
np.array(valid_source).shape

(32, 236)

In [126]:
with tf.Session(graph=train_graph) as sess:
    
    sess.run(tf.global_variables_initializer())
    train_writer1=tf.summary.FileWriter('./summary/train_loss', train_graph)
    train_writer2=tf.summary.FileWriter('./summary/valid_loss', train_graph)
    merge=tf.summary.merge_all()
    counter=0
    for epoch in range(epochs):
    
        state_fw=sess.run(fw_initial_state)
        state_bw=sess.run(bw_initial_state)
        
        for batch_i, (train_source, train_target) in enumerate(get_batches(source_letter_ids, target_letter_ids, source_letter_to_int['<PAD>'], target_letter_to_int['<PAD>'], batch_size)):
        
        
            feed={x:train_source,
                 y:train_target,
                 lr:learn_rate,
                 fw_initial_state:state_fw,
                 bw_initial_state:state_bw}
            
            counter+=1
        
            summary, cost, (state_fw, state_bw), _= sess.run([merge, loss, final_state, optm], feed)
            
            train_writer1.add_summary(summary, counter)
        
            if batch_i%display_step==0:
                
                feed={x:valid_source,
                     y:valid_target}

                summary, valid_loss=sess.run([merge, loss], feed)
                train_writer2.add_summary(summary, counter)
                
                print("Epoch{:>3}/{}   Batch{:>4}/{}   loss: {:>6.3f}  val_loss{:>6.3f}".format(
                                                epoch,
                                                epochs,
                                                batch_i,
                                                len(source_letter_ids)//batch_size,
                                                cost,
                                                valid_loss))
        
        saver.save(sess, "checkpoint/trained_weights12.ckpt")
    

Epoch  0/25   Batch   0/200   loss:  0.376  val_loss 0.328
Epoch  0/25   Batch 100/200   loss:  0.166  val_loss 0.133
Epoch  1/25   Batch   0/200   loss:  0.121  val_loss 0.122
Epoch  1/25   Batch 100/200   loss:  0.145  val_loss 0.118
Epoch  2/25   Batch   0/200   loss:  0.113  val_loss 0.113
Epoch  2/25   Batch 100/200   loss:  0.137  val_loss 0.111
Epoch  3/25   Batch   0/200   loss:  0.109  val_loss 0.108
Epoch  3/25   Batch 100/200   loss:  0.135  val_loss 0.107
Epoch  4/25   Batch   0/200   loss:  0.107  val_loss 0.105
Epoch  4/25   Batch 100/200   loss:  0.131  val_loss 0.105
Epoch  5/25   Batch   0/200   loss:  0.103  val_loss 0.102
Epoch  5/25   Batch 100/200   loss:  0.126  val_loss 0.101
Epoch  6/25   Batch   0/200   loss:  0.099  val_loss 0.099
Epoch  6/25   Batch 100/200   loss:  0.124  val_loss 0.099
Epoch  7/25   Batch   0/200   loss:  0.099  val_loss 0.098
Epoch  7/25   Batch 100/200   loss:  0.124  val_loss 0.098
Epoch  8/25   Batch   0/200   loss:  0.099  val_loss 0.0

KeyboardInterrupt: 

In [ ]:
"w10 0.133/0.119 without inter connection"
"w11 0.116/0.111 in epoch 15/25, 2layer 1 dropout 1 layer 1 dropout-> embed 1 layer16F - 32F dropout of 0.6 "

In [21]:
samples=test_seq_id[0]
targets=test_tar_id[0]

In [23]:
samples=source_letter_ids[9]
targets=target_letter_ids[9]

In [ ]:
def make_batches(sources, targets):
    for i in range(0, len(sources)):
        yield sources[i], targets[i]

In [22]:
with tf.Session(graph=train_graph) as sess:
    
    saver.restore(sess, tf.train.latest_checkpoint('checkpoint/'))
    print("\n")
    predict=sess.run(logits, {x:[samples]*batch_size})
    #print("Input Sequence : ", test_sor_[1], "\n")
    print("Sequence Length : ", len(samples), "\n")
    print("Actual Output Sequence : ", test_tar_[0], "\n")
    act=[sess.run(tf.argmax(val, axis=0)) for val in predict[0] ]
    print(act)
    score=sess.run(tf.reduce_mean(tf.cast(tf.equal(act, targets), tf.float32)))*100
    output_seq="".join(target_int_to_letter[i] for i in act if i != target_letter_to_int['<PAD>'])
    print("Generated Output Sequence : ", output_seq, "\n")
    print("Output seq length : ", len(output_seq), "\n")
    print("Prediction Score : ", score)

INFO:tensorflow:Restoring parameters from checkpoint/trained_weights12.ckpt


Sequence Length :  107 

Actual Output Sequence :  CHHHHHHHHHHHTTTGGGTTTCTTTTTTTCCCTTTTCCTTTTHHHHHHHHHHHHHHHHCCCHHHHHHHHHHHHTTTCHHHHHHHHHHHHHHCGGGTTTCCCCHHHHC 

[4, 4, 4, 3, 3, 2, 3, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 3, 3, 3, 3, 1, 4, 4, 4, 4, 1, 1, 4, 2, 2, 2, 2, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 4, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 3, 1, 1, 4, 1, 1, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3]
Generated Output Sequence :  CCCHHEHEEHHHHHHHHHHCCHHHHHHHTCCCCTTCEEEECCHHHHHHHHHHHHHHHHHTCHHHHHHHHHHHHHHHHHTCHHHHHHEEEEEHTTCTTCCCCCHHHHH 

Output seq length :  107 

Prediction Score :  55.14018535614014


In [63]:
#checking complete CB513
with tf.Session(graph=train_graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoint/'))
    print("\n")
    score=[]
    for samples, targets in get_batches(test_seq_id, test_tar_id, source_letter_to_int['<PAD>'], target_letter_to_int['<PAD>'],batch_size):
        act=[]
        predict=sess.run(logits, {x:samples})
        act.append([sess.run([tf.argmax(val, axis=0) for val in values]) for values in predict])
        act=np.array(act)
        act=act[0, :, :]
        targets=np.array(targets)
        score.append(sess.run(tf.reduce_mean(tf.cast(tf.equal(act, targets), tf.float32))))
    final_score=sess.run(tf.reduce_mean(score))*100
    print(score)
    print("Test Acc: ", final_score)

INFO:tensorflow:Restoring parameters from checkpoint/trained_weights12.ckpt


INFO:tensorflow:Restoring parameters from checkpoint/trained_weights12.ckpt




[0.767214, 0.7962677]
Tensor("mul:0", shape=(), dtype=float32)


In [ ]:
   print("\n")
    print(act, "\t", len(act))
    print("\n")
    output_seq="".join(target_int_to_letter[i] for i in act if i != target_letter_to_int['<PAD>'])
    print(output_seq, "\t", len(output_seq))
    print("\n")
    print("Prediction Score : ", )